Las siguientes funciones fueron creadas para valorar opciones por el método de Cox-Rubinstein (árboles binomiales), Montecarlo y Black-Scholes

In [1]:
#Librerias
import math
import numpy as np
from scipy import stats
import scipy.stats

###Funciones

In [2]:
#funcion arbol binomial call
def call_arbinomial(So, Strike, r, vol, t, nodos):
  #Calculo parametros
  plazo_año = t/360
  delta_t = plazo_año/nodos
  u = np.exp(vol*np.sqrt(delta_t))
  d = 1/u
  p = (np.exp(r*delta_t)-d)/(u-d)
  p_1 = 1- p
  f_dto = np.exp(-r*delta_t)

  #Arbol de precios

  #Matriz inicial en cero
  matriz_precios = np.zeros((nodos + 1, nodos +1))
  #Precio Spot [1,1]
  matriz_precios[0,0] = So
  #Fila Up
  for i in range(1, nodos + 1):
    matriz_precios[0,i] = matriz_precios[0,i-1] * u
  #Fila Down
  for i in range(1, nodos + 1):
    for j in range(i, nodos + 1):
      matriz_precios[i,j] = matriz_precios[i-1,j-1] * d

  #Arbol de Payoff
  #matriz en ceros
  matriz_payoff = np.zeros((nodos + 1, nodos +1))
  #payoff St
  for i in range(nodos + 1):
    matriz_payoff[i,-1] = max(matriz_precios[i,-1] - Strike, 0)

  #Inducción hacia atras
  for j in range(nodos-1, -1, -1):
    for i in range(j+1):
      matriz_payoff[i, j] = ((matriz_payoff[i, j + 1] * p) + (matriz_payoff[i + 1, j + 1] * p_1)) * f_dto

  print("La prima de la opción es: ", matriz_payoff[0,0])

In [22]:
#Put americana
def put_arbinominal(So, Strike, r, vol, t, nodos):
    #Cálculo de parámetros
    plazo_año = t / 360
    delta_t = plazo_año / nodos
    u = np.exp(vol * np.sqrt(delta_t))
    d = 1 / u
    p = (np.exp(r * delta_t) - d) / (u - d)
    p_1 = 1 - p
    f_dto = np.exp(-r * delta_t)

    #Arbol de precios
    matriz_precios = np.zeros((nodos + 1, nodos + 1))
    matriz_precios[0, 0] = So
    for i in range(1, nodos + 1):
        matriz_precios[0, i] = matriz_precios[0, i - 1] * u
    for i in range(1, nodos + 1):
        for j in range(i, nodos + 1):
            matriz_precios[i, j] = matriz_precios[i - 1, j - 1] * d

    #Arbol Payoff
    matriz_payoff = np.zeros((nodos + 1, nodos + 1))
    for j in range(nodos, -1, -1):
        for i in range(j + 1):
            matriz_payoff[i, j] = max(Strike - matriz_precios[i, j], 0)

    #Arbol valoración
    matriz_put = np.zeros((nodos + 1, nodos + 1))

    #La última columna de matriz_put es el payoff de una put
    for i in range(nodos + 1):
        matriz_put[i, -1] = max(Strike - matriz_precios[i, -1], 0)

    #Llenar matriz_put con inducción hacia atrás
    for j in range(nodos - 1, -1, -1):
        for i in range(j + 1):
            matriz_put[i, j] = max(matriz_payoff[i, j], ((matriz_put[i, j + 1] * p) + (matriz_put[i + 1, j + 1] * p_1)) * f_dto)

    print("La prima de la opción Put americana es: ", matriz_put[0, 0])

In [3]:
def put_arbinomial_eur(So, Strike, r, vol, t, nodos):
  #Calculo parametros
  plazo_año = t/360
  delta_t = plazo_año/nodos
  u = np.exp(vol*np.sqrt(delta_t))
  d = 1/u
  p = (np.exp(r*delta_t)-d)/(u-d)
  p_1 = 1- p
  f_dto = np.exp(-r*delta_t)

  #Arbol de precios

  #Matriz inicial en cero
  matriz_precios = np.zeros((nodos + 1, nodos +1))
  #Precio Spot [1,1]
  matriz_precios[0,0] = So
  #Fila Up
  for i in range(1, nodos + 1):
    matriz_precios[0,i] = matriz_precios[0,i-1] * u
  #Fila Down
  for i in range(1, nodos + 1):
    for j in range(i, nodos + 1):
      matriz_precios[i,j] = matriz_precios[i-1,j-1] * d

  #Arbol de Payoff
  #matriz en ceros
  matriz_payoff = np.zeros((nodos + 1, nodos +1))

  #payoff put
  for i in range(nodos + 1):
    matriz_payoff[i,-1] = max(Strike - matriz_precios[i,-1], 0)

  #Inducción hacia atras
  for j in range(nodos-1, -1, -1):
    for i in range(j+1):
      matriz_payoff[i, j] = ((matriz_payoff[i, j + 1] * p) + (matriz_payoff[i + 1, j + 1] * p_1)) * f_dto

  print("La prima de la opción es: ", matriz_payoff[0,0])

In [5]:
#función para valorar opciones usando Montecarlo
def montecarlo(So, Strike, r, vol, t, n):
  plazo_año = t/360
  matriz_st = np.zeros(n)
  matriz_payoff = np.zeros((n,2))

  #Cálculo precio
  for i in range(1, n):
      St = So * np.exp((r - (vol**2) / 2) * plazo_año + vol * np.sqrt(plazo_año) * scipy.stats.norm.ppf(np.random.rand()))
      matriz_st[i] = St

      #Payoff Call
      payoff_call = max(St - Strike, 0)
      matriz_payoff[i, 0] = payoff_call

      #Payoff Put
      payoff_put = max(Strike - St, 0)
      matriz_payoff[i, 1] = payoff_put

  #Payoff en T
  call_T = np.average(matriz_payoff[:,0])
  put_T = np.average(matriz_payoff[:,1])

  #Payoff en t
  call_t = call_T * np.exp(-r*plazo_año)
  put_t = put_T * np.exp(-r*plazo_año)

  print("La prima de la opción Call es: " + str(call_t) + " y la prima de la opción Put es: " + str(put_t))

In [6]:
#Función de valoración via Black-Scholes
def BlackScholes(So, Strike, r, vol, t):
  plazo_año = t/365
  d1 = (np.log(So/Strike) + (r + (vol**2)/2) * plazo_año)/(vol* np.sqrt(plazo_año))
  d2 = (np.log(So/Strike) + (r - (vol**2)/2) * plazo_año)/(vol* np.sqrt(plazo_año))

  #Call
  call = np.exp(-r*plazo_año) * (So*stats.norm.cdf(d1)*np.exp(r*plazo_año) - Strike*stats.norm.cdf(d2))

  #Put
  put = np.exp(-r*plazo_año) * (Strike*stats.norm.cdf(-d2) - So*stats.norm.cdf(-d1)*np.exp(r*plazo_año))

  print("La prima de la opción Call es: " + str(call) + " y la prima de la opción Put es: " + str(put))

Punto 1 | Call Europea

In [7]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4000
Precio strike: 4400
Tasa de interes (continua): 0.065
Volatilidad: 0.17
Tiempo (días): 90
Nodos: 12


In [8]:
call_arbinomial(So, Strike, r, vol, t, nodos)

La prima de la opción es:  31.45095252891365


In [9]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 33.3029604824784 y la prima de la opción Put es: 362.5674791074205


In [10]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 32.801597683621104 y la prima de la opción Put es: 362.84317302573805


Punto 2 | Call Americana

In [11]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4000
Precio strike: 4000
Tasa de interes (continua): 0.065
Volatilidad: 0.21
Tiempo (días): 150
Nodos: 50


In [12]:
call_arbinomial(So, Strike, r, vol, t, nodos)

La prima de la opción es:  269.8516824866253


In [14]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 269.8054203456276 y la prima de la opción Put es: 165.3717386823794


In [15]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 268.70298398146923 y la prima de la opción Put es: 163.26814330830993


Punto 3 | Put Americana

In [40]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4000
Precio strike: 4060.45
Tasa de interes (continua): 0.06
Volatilidad: 0.22
Tiempo (días): 360
Nodos: 100


In [41]:
put_arbinominal(So, Strike, r, vol, t, nodos)

La prima de la opción Put americana es:  291.49193691090727


In [42]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 436.9019404915758 y la prima de la opción Put es: 260.9065511735067


In [43]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 433.78612554571157 y la prima de la opción Put es: 260.91822160872175


Punto 4 | Put Europea

In [44]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4000
Precio strike: 4050
Tasa de interes (continua): 0.055
Volatilidad: 0.19
Tiempo (días): 120
Nodos: 40


In [45]:
put_arbinomial_eur(So, Strike, r, vol, t, nodos)

La prima de la opción es:  163.78152405868877


In [46]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 186.61348226856094 y la prima de la opción Put es: 162.84409795581732


In [47]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 184.79190768064785 y la prima de la opción Put es: 162.2171636317979


Punto 5 | Call Europea

In [48]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4050
Precio strike: 4020
Tasa de interes (continua): 0.0625
Volatilidad: 0.18
Tiempo (días): 120
Nodos: 40


In [49]:
call_arbinomial(So, Strike, r, vol, t, nodos)

La prima de la opción es:  228.3607143233863


In [50]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 227.17377106525953 y la prima de la opción Put es: 115.25009167787643


In [51]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 226.26167291914624 y la prima de la opción Put es: 114.50180848760765


Punto 6 | Call Europea

In [52]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4050
Precio strike: 4111.21
Tasa de interes (continua): 0.0675
Volatilidad: 0.17
Tiempo (días): 360
Nodos: 100


In [53]:
call_arbinomial(So, Strike, r, vol, t, nodos)

La prima de la opción es:  384.0109151789004


In [54]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 383.4126598142115 y la prima de la opción Put es: 175.6143961092718


In [55]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 379.66914832619483 y la prima de la opción Put es: 176.08607517097434


Punto 7 | Put Europea

In [56]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4050
Precio strike: 4075
Tasa de interes (continua): 0.07
Volatilidad: 0.15
Tiempo (días): 90
Nodos: 12


In [59]:
put_arbinomial_eur(So, Strike, r, vol, t, nodos)

La prima de la opción es:  98.81543577691117


In [58]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 144.9728915878707 y la prima de la opción Put es: 98.59897511269305


In [60]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 143.34673187305032 y la prima de la opción Put es: 98.61464409417248


Punto 8 | Put Americana

In [61]:
#Parametros iniciales
So=float(input("Precio spot: "))
Strike=float(input("Precio strike: "))
r=float(input("Tasa de interes (continua): "))
vol=float(input("Volatilidad: "))
t=float(input("Tiempo (días): "))
nodos=int(input("Nodos: "))

Precio spot: 4050
Precio strike: 4050
Tasa de interes (continua): 0.0725
Volatilidad: 0.195
Tiempo (días): 150
Nodos: 50


In [62]:
put_arbinominal(So, Strike, r, vol, t, nodos)

La prima de la opción Put americana es:  157.30533078033955


In [63]:
montecarlo(So, Strike, r, vol, t, 150000)

La prima de la opción Call es: 267.25963885863706 y la prima de la opción Put es: 144.67527059902835


In [64]:
BlackScholes(So, Strike, r, vol, t)

La prima de la opción Call es: 263.9400958702114 y la prima de la opción Put es: 145.05218651638089
